In [1]:
# This allows multiple outputs from a single jupyter notebook cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

# 第35章 OBV 指標交易策略

## 35.1 OBV 指標概念

OBV 的英文全稱是 On Balance Volume，中文為能量潮，是由美國的投資解析家 Joe Granville 在 20 世紀 60 年代所創的一種技術指標，他認為市場的動能應該由成交量的變化情況來反映。成交量可以反映出市場買賣雙方的活躍情況，量是價的先行者，價格的變化與成交量有密切關係。OBV 指標從量入手來對價格的走勢來作出預測，將成交量指標化，製成趨勢線，配合股價趨勢線，從價格的變動及成交量的增減關係來推測股價變動趨勢。

## 35.2 OBV 指標計算方法

對於能量潮 OBV 的指標計算，一般有累積 OBV、移動 OBV，和成交量多空比率淨額這三種計算方式。

1. 累積 OBV

OBV 主要計算累積成交量，將股價上漲時的成交量進行正累加，股價下跌時的成交量進行負累加。其計算公式為：

$$\mathit{OBV}_n=\pm V_m+\mathit{OBV}_{n-1}.$$

其中 $\mathit{OBV}_{n}$ 和 $\mathit{OBV}_{n-1}$ 分別是本期和前一期的 OBV 值、而 $V_n$ 則是當日的成交量。

* 當本期股價上漲時，$V_n$ 的象黴式為正，$\mathit{OBV}_{n}=\mathit{OBV}_{n-1}+V_n$

* 當本期股價下跌時，$V_n$ 的象徵式則為負，$\mathit{OBV}_{n}\mathit{OBV}_{n-1}-V_n$

以聯發科股票的交易資料為分析對象，計算 OBV 的值。Python 程式碼如下：

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
MediaTek = pd.read_csv('.\\PythonBook_code_data\\part5\\35\\2454.csv',sep='\t')
MediaTek.index = pd.to_datetime(MediaTek.Date)
close=MediaTek.Close
Volume=MediaTek.Volume
# 計算OBV
difClose=close.diff()
difClose[0]=0
OBV=(((difClose>=0)*2-1)*Volume).cumsum()
OBV.name='OBV'
OBV.head()
OBV.describe()

Date
2015-01-05    2482
2015-01-06   -3443
2015-01-07    9699
2015-01-08    1017
2015-01-09    8762
Name: OBV, dtype: int64

count       488.000000
mean      41374.520492
std       80542.372868
min     -123158.000000
25%      -17326.250000
50%       25741.000000
75%      122761.500000
max      193366.000000
Name: OBV, dtype: float64

2. 移動型 OBV

移動型 OBV 是由累積 OBV 進行簡單移動平均得到，一般選取 9 天或者 12 天為時間跨度，移動型 OBV 的計算公式為：

$$\mathrm{smOBV}_t=\frac{\mathrm{OBV}_{t}+\mathrm{OBV}_{t-1}+...+\mathrm{OBV}_{t-8}}{9},t=9,10,...$$

用 Python 計算聯發科股票的移動型 OBV 值。

In [4]:
# 計算移動型OBV
#import movingAverage as mv
smOBV=mv.smaCal(OBV,9)
smOBV.tail()

NameError: name 'mv' is not defined

3. 修正型 OBV

此外，在計算累積成交量時，無論股價變化幅度與趨勢，當期的成交量的權重一樣；若要涵蓋價格變化幅度的訊息，一般用多空比率淨額（Volume Accumulation）來交替單純的成交量。多空比率淨額的計算公式為：

$$\mathit{VA}_n=\mathit{VA}_{n-1}+V_n\times\frac{(C_n-L_n)-(H_n-C_n)}{H_n-L_n}$$

其中，$V_n$ 為當日成交量，而，$H_n$、$L_n$、$C_n$，則分別是當日的最高價、最低價和收盤價。收盤價與最低價的差值表示多頭力量的強度，最高價與收盤價的差值表明空頭力量的強度，兩者之差表示多頭的淨力量幅度。用這兩個差的差值 $(C_n-L_n)-(H_n-C_n)$，與最高價與最低價的差值，$H_n-L_n$ 之比，表示多頭相對力量對於成交量的貢獻程度。

接下來，用 Python 計算聯發科股票的修正型 OBV 值。

In [ ]:
# 計算修正型OBV
AdjOBV=((close-MediaTek.Low)-(MediaTek.High-close))/(MediaTek.High-MediaTek.Low)*Volume
AdjOBV.name='AdjOBV'
AdjOBV.head()
AdjOBVd=AdjOBV.cumsum()
AdjOBVd.name='AdjOBVd'
AdjOBVd.describe()
# 繪製能量潮線圖
ax1.plt.subplot(3,1,1)
close.plot(title='聯發科收盤價')
plt.xticks(close.index[1:3],(''))
plt.xlabel('')
ax2=plt.subplot(3,1,2)
OBV.plot(label='OBV',title='聯發科累積能量潮與移動能量潮')
smOBV.plt(label='smOBV',linestyle='-.',color='r')
plt.xlable('')
plt.legend(loc='upper left')
plt.xticks(close.index[1:3],(''))
ax3=plt.subplot(3,1,3)
AdjOBVd.plot(title='成交量多空比率淨額')
plt.show()

圖 35.1.收盤價與3種OBV曲線圖

## 35.3 OBV 指標的理論依據

在探討如何應用 OBV 指標進行交易策略編寫之前，我們先來談一下 OBV 指標的原理。

當投資者對股價的預期不一致時，成交量往往會較大；而當投資者對股價的預期一致時，成交量的取值一般會較小。例如，當投資者都認為股價即將上漲時，買單的數量就會變多。持有股票的人卻不會賣出，賣單的數量就會很少，進而成交量也變少。反之，當一部分人預期股價上漲，另一部分預期股價下跌時，市場上的買單和賣單會維持一個較高的數量，進而使得成交量上升。

根據物理學上的重力原理，物體不會一直上升，總會下跌，而且物體在上升時所需要的能量要比下降時要多。當我們把這個原理類比於股市時，會總結出一個量與價的關係：股價易跌難漲；而且股價下跌所需成交量可能小於股價上升所需成交量。

根據慣性原理的思路，熱門股在相當的一段時間內都會保持較大的成交量和價格波動，而冷門股則相反。

實際上，成交量代表著買賣雙方的交易總數，而股票的價格正是在這些交易中形成的。因此，在解析股價的時候，成交量與股價之問的關係不容忽視。

## 35.4 OBV 指標的交易策略制定

股價的趨勢可以透過成交量的多少來觀察，OBV 指標透過成交量的多少來體現市場的能量。下面設計一個簡單的 OBV 指標交易策略：

當 OBV 指標増大時，說明累積成交量在增加，可以推測當期的價格變化為正值，當期的股價處於上升狀態；累積成交量的增大也體現出市場活躍度在増加，短期內，股票價格有可能繼續上升，釋放出買入訊號。

當 OBV 指標變小時，說明累積成交量在減小，當期的價格變化為負值，當期股價是下跌的，市場的活躍度减弱；短期內，股價有可能還會下跌，釋放出賣出訊號。

## 35.5 OBV 指標交易策略的 Python 實測

按上述 OBV 指標交易策略的思路，運用 Python 進行交易實測。交易策略的思路大致如下：

* 構造交易函數 trade(obv,price)，依照指標的取值情況確定買賣點訊號 signal，進而根據 signal 計算交易策略的的收益率；

* 構造交易表現函數 backtest(ret,tradeRet)，繪製累積收益率曲線圖，計算年化收益率、夏普比率等。

首先，我們用累積 OBV 值作為 OBV 指標的取值。

In [ ]:
# 定義交易策略函數
import ffn
def trade(obv,price):
    signal=(2*(obv.diff()>0)-1)[1:]
    ret=ffn.to_returns(price)[1:]
    ret.name='ret'
    tradeRet=ret*signal.shift(1)
    tradeRet.name='tradeRet'
    Returns=pd.merge(pd.DataFrame(ret),\
                     pd.DataFrame(tradeRet),\
                     left_index=True,right_index=True).dropna()
    return(Returns)
# OBV指標交易策略
OBVtrade=trade(OBV,close)
OBVtrade.head()
# 評價交易策略表現
ret=OBVtrade.ret
tradeRet=OBVtrade.tradeRet
ret.name='BuyAndHold'
tradeRet.name='OBVTrade'
(1+ret).cumprod().plt(label='ret',linestyle='dashed')
(1+tradeRet).cumprod().plt(label='tradeRet')
plt.title('累積OBV交易策略績效表現')
plt.legend()
plt.show()
# 定義交易表現函數
def backtest(ret,tradeRet):
    def performance(x):
        winpct=len(x[x>0])/len(x[x!=0])
        annRet=(1+x).cumprod()[-1]**(245/len(x))-1
        sharpe=ffn.calc_risk_return_ratio(x)
        maxDD=ffn.calc_max_drawdown((1+x).cumprod())
        perfo=pd.Series([winpct,annRet,sharpe,maxDD],index=['win rate','annualized return','sharpe ratio','maximum drawdown'])
        return(perfo)
    BuyAndHold=performance(ret)
    OBVTrade=performance(tradeRet)
    return(pd.DataFrame({ret.name:BuyAndHold,\
            tradeRet.name:OBVTrade}))
# OBV 指標交易表現
OBVtest=backtest(ret,tradeRet)
OBVtest

圖 35.2：累積OBV交易策略耫效表現圖

接著，用移動 OBV 值作為 OBV 指標的取值。

In [ ]:
smOBVtrade=trade(smOBV,close)
smOBVtrade.head(n=3)
ret=smOBVtrade.ret
ret.name='BuyAndHold'
smtradeRet=smOBVtrade.tradeRet
(1+ret).cumprod().plot(label='ret',linestyle='dashed')
(1+tradeRet).cumprod().plot(label='tradeRet')
plt.title('移動OBV交易策略績效表現')
plt.legend()
plt.show()
test=backtest(ret,smtradeRet)
test

圖 35.3：移動OBV交易策略績效表現圖

## 35.6 OBV 指標的應用原則

根據 OBV 指標的原理，股價的趨勢是透過成交量的多少來體現。成交量的多與少是一種相對概念，將 OBV 值與過去的 OBV 值進行比對，才能對成交量的多少做出更合理的判斷。觀察 OBV 曲線與股價的趨勢，對未來的價格變動做出較為客觀的預測。在使用 OBV 指標時應該注意幾個原則：

* OBV 曲線上升但是股價下降：表明在股價下跌時仍有成交量卻在上升，逐漸提高的成交量表明投資者對該股的信心不斷增強，釋放出買進的訊號。

* OBV 曲線下降而股價上漲：股價仍在上升，成交量卻在縮小，表明投資者對其逐漸喪失信心，該股很可能已接近頂點，即將下跌。

* OBV 曲線穩步上升，同時股價上漲：表明行情穩步向上，股價仍有上漲空問，中長期走勢良好。

* OBV 曲線緩慢下降，同時股價下跌：則表明行情不佳，股價仍會繼續下跌，投資者應賣出股票或暫時觀望。

* 兩種極端情況：OBV 曲線快速上升的現象表明買盤迅速湧入，持續性不強，股價在短暫的拉升後可能會迅速下跌；OBV 曲線快速下降的現象則表明賣盤迅速湧入，但是隨後仍有可能會有一段較長的下跌，投資者應以觀望為主。

上面介紹的只是使用 OBV 指標時，所應參照的幾點原則，實際使用時依據這些原則不一定會獲利。本書建議讀者結合其他指標綜合解析，而不是僅依靠單一指標，對行情的研判才能更加周延。